In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta

from Tools import *
from Tools2 import *
import numpy as np

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
yf.pdr_override()

In [2]:
from itertools import combinations

lst = [5, 6, 11,12]
index=[]
i=0
while i<len(lst):
    for comb in combinations(lst,r=i):
        if comb==():
            continue
        index.append(list(comb))
    i+=1
print(index)
        


[[5], [6], [11], [12], [5, 6], [5, 11], [5, 12], [6, 11], [6, 12], [11, 12], [5, 6, 11], [5, 6, 12], [5, 11, 12], [6, 11, 12]]


In [3]:
start="2020-12-31"
end="2022-12-31"
name="AAPL"

#MACD619RSI21_8020
stock_data=pct_change(name,start,end)
#stock_data=data.get_data_yahoo(tickers=name,start=start,end=end,progress=False)
    
stock_data=MACD_short(stock_data)
stock_data=RSI(stock_data)
stock_data=RSI_MACD(stock_data,21,80,20)


stock_data = stock_data.dropna()
del stock_data["real_situation"]
del stock_data["RSI6"]
del stock_data["RSI14"]

y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

high=0
for i in index:
    X = stock_data.iloc[:,i]
    split = int(0.7*len(stock_data))
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    model = LogisticRegression()
    model = model.fit (X_train,y_train)
    probability = model.predict_proba(X_test)
    predicted = model.predict(X_test)
    
    cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
    mean_score=cross_val.mean()
    print(mean_score)
    if mean_score>=high:
        high=mean_score
        print(high)
        print(i)
    




0.5010204081632652
0.5010204081632652
[5]
0.503061224489796
0.503061224489796
[6]
0.5008928571428571
0.44272959183673477
0.5010204081632652
0.5010204081632652
0.5010204081632652
0.5008928571428571
0.43647959183673474
0.4801020408163265
0.5010204081632652
0.5010204081632652
0.5010204081632652
0.4801020408163265


In [4]:
test

NameError: name 'test' is not defined

In [ ]:
y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

In [ ]:
split = int(0.7*len(stock_data))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [ ]:
model = LogisticRegression()
model = model.fit (X_train,y_train)

In [ ]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

In [ ]:
probability = model.predict_proba(X_test)
print(probability)

predicted = model.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, predicted))

In [ ]:
print(metrics.classification_report(y_test, predicted))

In [ ]:
print(model.score(X_test,y_test))
#0.4689655172413793 纯volume

In [ ]:
cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print(cross_val)
print(cross_val.mean())

In [ ]:
stock_data['Predicted_Signal'] = model.predict(X)
stock_data['Nifty_returns'] = np.log(stock_data['Close']/stock_data['Close'].shift(1))
Cumulative_Nifty_returns = np.cumsum(stock_data[split:]['Nifty_returns'])

stock_data['Startegy_returns'] = stock_data['Nifty_returns']* stock_data['Predicted_Signal'].shift(1)
Cumulative_Strategy_returns = np.cumsum(stock_data[split:]['Startegy_returns'])

plt.figure(figsize=(10,5))
plt.plot(Cumulative_Nifty_returns, color='r',label = 'Nifty Returns')
plt.plot(Cumulative_Strategy_returns, color='g', label = 'Strategy Returns')
plt.legend()
plt.show()